
# 3.2 字典的现代句法
## 3.2.1 字典推导式
    

In [ ]:
dial_codes = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]

country_code = {country: code for code, country in dial_codes}
print(country_code)

print({code: country.upper() for country, code in country_code.items() if code < 66})

## 3.2.2 映射拆包

In [ ]:
def dump(**kwargs):
    print(kwargs)

print(dump(**{'x':1}, y=2, **{'z':3}))

print({'a':0, **{'b':1}, **{'c':2, 'b':3}})

## 3.2.3 使用|合并映射

In [ ]:
d1 = {'a':1, 'b':2}
d2 = {'b':3, 'c':4}

print(d1 | d2)
# 通常合并的新map的类型与左边的map类型相同

# 就地更新
d1 |= d2
print(d1)

# 3.3 使用模式匹配处理映射
    match/case语句的匹配对象可以是映射，能匹配collections.abc.Mapping的任何具体子类或虚拟子类


In [ ]:
def get_creators(record:dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'authors': name}:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"Missing 'authors' field: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            raise ValueError(f"Unsupported record: {record!r}")
        
b1 = dict(type='book', api=2, authors=['Aldous Huxley'], title='Brave New World')
print(get_creators(b1))

from collections import OrderedDict
# 模式中键的顺序不重要
b2 = OrderedDict(type='book', api=2, authors=['Aldous Huxley'], title='Brave New World')
print(get_creators(b2))

与序列模式匹配不同的是，map只要部分匹配就算成功匹配。  
可以使用**变量 来捕获剩余的键值对，不过必须放在模式的最后。

In [ ]:
food = dict(category='ice cream', flavor='vanilla', cost=199)

match food:
    case {'category': 'ice cream', **details}:
        print(f"Ice cream details: {details}")

# 3.4 映射类型的标准API

In [ ]:
import collections.abc as abc

my_dict = {'a':1, 'b':2, 'c':3}
print(isinstance(my_dict, abc.Mapping))
print(isinstance(my_dict, abc.MutableMapping))

想要自定义map类型，可以扩展collections.UserDict类或者通过组合模式包装dict。 键必须可哈希。

## 3.4.1 “可哈希”指什么
    如果一个对象的哈希码在整个生命周期中都不改变并且可以与其他对象进行比较，那么这个对象就是可哈希的。两个可哈希对象仅当哈希码相同时相等。

In [ ]:
tt = (1, 2, (30, 40))
print(hash(tt))

# tuple包含可变对象list所以不可hash
tl = (1, 2, [30, 40])
# print(hash(tl))

tf = (1, 2, frozenset([30, 40]))
print(hash(tf))

## 3.4.3 插入或更新可变值
    使用setdefault方法，如果键不存在，就插入键值对，如果存在，就返回对应的值，不做修改。

In [ ]:
d = {'a': 1}

print(d.get('b', 2))
print(d)

print(d.setdefault('b', 2))
print(d)

# 3.5 自动处理缺失的键
    有时搜索的键不一定存在，可以人为设置一个值，有两种办法：第一种是使用defaultdict类，第二种是实现__missing__方法。
## 3.5.1 defaultdict：处理缺失键的另一种选择

In [ ]:
from collections import defaultdict

# dict[new_key] 语法会调用 list() 来生成默认值, 然后插入到字典中对应new_key的位置, 最后返回这个默认值
dd = defaultdict(list)
print(dd.get("a"))
print(dd["a"])
print(dd.get("a"))

## 3.5.2 __missing__方法
    如果dict的子类定义了这个方法，那么dict.__getitem__找不到键时将会调用它，而不是抛出一个KeyError异常。

In [ ]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    # 未使用 k in dict语法是因为这样会导致 __contains__ 递归调用
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

d = StrKeyDict0([('2', 'two'), ('4', 'four')])
print(d['2'])

print(d[4])
# print(d[1])
print(d.get(1, 'N/A'))

## 3.5.3 标准库对__missing__方法的使用不一致
- dict子类
    定义一个dict子类，只实现__missing__方法，这种情况下只有d\[k\]语法会调用__missing__方法。
- collections.UserDict子类
    定义一个UserDict子类，只实现__missing__方法，这种情况下d\[k\]和d.get(k)都会调用__missing__方法。
- abc.Mapping子类
    如果实现的__getitem__方法不调用__missing__方法，那么这个类永不触发__missing__方法。
- abc.Mapping子类
    如果实现的__getitem__方法调用__missing__方法，d\[k\]、d.get(k)和k in d这些操作就会触发__missing__方法。

# 3.6 dict的变体
## 3.6.1 collections.OrderedDict

    从python3.6起，dict类型也是有序的。不过dict和OrderedDict有一些差异。

- OrderedDict的相等检查考虑顺序
- OrderedDict的popitem方法可通过一个可选参数指定移除哪一项。
- OrderedDict多了一个move_to_end方法可以将任一现有键移动到另一端（默认是末尾，也可以是开头）。
- 常规的dict顺序是次要的。
- OrderedDict的目的是方便执行重新排序操作，性能是次要的。
- OrderedDict处理频繁重新排序操作比dict好。

## 3.6.2 collections.ChainMap
    ChainMap实例存放一组映射，可作为一个整体来搜索，然后在进行键查找操作时，会按照输入顺序查找，查找到就结束。chainMap存放的是map的引用。

In [ ]:
d1 = dict(a=1, b=3)
d2 = dict(a=2, b=4, c=6)
from collections import ChainMap
chain = ChainMap(d1, d2)
print(chain['a'])
print(chain['c'])

# chainMap的更新或者插入操作只影响第一个输入的map
chain['c'] = -1
print(chain['c'])
print(d1['c'])
print(d2['c'])

## 3.6.3 collections.Counter
    Counter是一种对键进行计数的映射。更新现有键，计数增加。可用于统计可哈希对象的实例数量，或者作为多重集（multiset）。Counter实现了+和-运算符，可以方便地进行合并。

In [ ]:
import collections
ct = collections.Counter('abracadabra')
print(ct)
ct.update('aaaaazzz')
print(ct)
print(ct.most_common(2))

## 3.6.4 shelve.Shelf
    标准库中的shelve模块持久存储字符串键与（以pickle二进制格式序列化的）Python对象之间的映射。
    感觉用处不大不做笔记赘述。

## 3.6.5 子类应继承UserDict而不是dict
    原因是内置的dict在实现上走了一些捷径，如果继承dict，那就不得不覆盖一些方法。
    UserDict没有继承dict，内部有一个dict实例，名为data。

In [ ]:
import collections
class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

UserDict拓展abc.MutableMapping。有两个方法需要注意。
MutableMapping.update方法使用self\[k\] = v这种方式插入新值，这样会调用__setitem__方法，而不是直接更新self.data。
Mapping.get方法默认通过__getitem__方法来获取值，这样就能触发__missing__方法了。

# 3.7 不可变映射
    标准库中提供的映射类型都是可变的，不过可以通过变通方案。
    types模块中有一个封装类名为MappingProxyType，可以返回一个只读的映射视图。

In [ ]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)
print(d_proxy[1])
try:
    d_proxy[2] = 'x'
except TypeError as e:
    print(e)

d[2] = 'B'
print(d_proxy)
print(d_proxy[2])

# 3.8 字典视图
    dict的实例方法.keys()、.items()和.values()分别返回dict_keys、dict_items和dict_values类的实例。
    这些字典视图是dict内部实现使用的数据结构的只读投影。

In [ ]:
d = dict(a=10, b=20, c=30)
values = d.values()
print(values)
print(len(values))
print(list(values))
print(reversed(values))
# reversed返回视图的一个反向迭代器
for v in reversed(values):
    print(v)

try:
    print(values[0])
except TypeError as e:
    print(e)

# 视图动态变化
d['z'] = 100
print(d)
print(values)

dict_keys、dict_items 和 dict_values 是内部类，可以得到实例但不能手动创建。

# 3.9 dict 的实现方式对实践的影响

    Python使用哈希表实现dict。效率很高但是会有些影响。
    - 键必须是可哈希的对象。
    - 通过键访问项的速度非常快。
    - Python3.6开始保留键的顺序。
    - dict占用大量内存。
    - 为了节省内存，不要在__init__方法之外创建实例属性。原因是实例属性会被存储在一个名为__dict__的字典中，所有实例共享，如果一个新的实例在__init__完之后添加了新的属性，Python就不得不为这个实例的__dict__创建一个新的哈希表。

# 3.10 集合论

    集合代指set和frozenset两种类型。集合是一组唯一的对象。集合的基本作用是去除重复项。集合的元素必须可哈希。set类型不可哈希。frozenset类型可哈希,所以可以作为set的元素。


In [ ]:
l = ['spam', 'spam', 'eggs', 'spam']
print(set(l))
print(list(set(l)))

集合类型通过中缀运算符实现了许多集合运算。
a | b 计算并集。 a & b 计算交集。 a - b 计算差集。 a ^ b 计算对称差集。

## 3.10.1 set字面量
    set字面量的句法很像数学中的集合句法，不过必须使用set()函数来创建空集合。

In [ ]:
s = {1}
print(s)
print(type(s))

s.pop()
print(s)

fronzenset没有字面量句法，必须调用构造函数来创建。

In [41]:
frozenset([1, 2, 3])

frozenset({1, 2, 3})

## 3.10.2 集合推导式

In [ ]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

# 3.11 集合的实现方式对实践的影响
    set和frozenset类型都使用哈希表实现。
    - 集合里的元素必须是可哈希的。
    - 集合成员测试效率很高。
    - 集合的内存开销大。
    - 元素的存储顺序取决于添加顺序。
    - 往集合里添加元素，可能会改变集合里已有元素的顺序。

集合运算以及字典视图的集合运算，见书。举个例子。

In [46]:
# 快速计算两个dict都有的键
d1 = {'a': 1, 'b': 2, 'c': 3}
d2 = {'b': 2, 'c': 3, 'd': 4}
print(d1.keys() & d2.keys())

# 字典视图兼容set实例
s = {'a', 'i'}
print(d1.keys() & s)
print(d1.keys() | s)

{'b', 'c'}
{'a'}
{'b', 'i', 'a', 'c'}
